In [ ]:
import requests
import pandas as pd
import json
import os
import sys
from datetime import datetime, timedelta
import csv

from dotenv import load_dotenv
import os


In [ ]:
load_dotenv()


In [ ]:
url = "https://data.alpaca.markets/v1beta1/news"
headers = {
    'Apca-Api-Key-Id': os.getenv('ALPACA_API_KEY'),
    'Apca-Api-Secret-Key': os.config('ALPACA_API_SECRET')
}


In [ ]:
# Stocks :- AAPL, MSFT, AMZN, NVDA, TSLA, GOOGL, UNH
# Sector Indices :- SPINF (AAPL,MSFT,NVDA,AVGO,ADBE,CRM,CSCO,ACN,AMD,ORCL)

ticker = "AAPL"     


start_date = datetime(2012, 1, 1)
end_date = datetime(2022, 12, 31)

In [ ]:

# Extract news details into a list
news_list = []

current_date = start_date
while current_date <= end_date:

  querystring = {
    'start': f"{current_date.date()}T00:00:01Z",
    'end': f"{current_date.date()}T23:59:59Z",
    'symbols': ticker,
    'limit': 50,
    # 'include_content': True,
    # 'exclude_contentless': True
    'include_content': False,
    'exclude_contentless': False,
  }

  response = requests.request(
    "GET", url, headers=headers, params=querystring)

  data = response.json()



  for index, news in enumerate(data["news"]):
    headline = news["headline"]
    summary = news["summary"]
    # content = news["content"]
    date = datetime.fromisoformat(news["created_at"].rstrip('Z')).date()

    news_list.append([date, headline, summary])

  current_date += timedelta(days=1)


In [ ]:

directory = "RawArticles"

#If directory doesn't exist, create the directory
if not os.path.exists(directory):
    os.makedirs(directory)

filename = f"{directory}/{ticker}_news_data.csv"


In [ ]:
# Sort the news list by date
news_list.sort(key=lambda x: x[0])

with open(filename, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['Date', 'Headline', 'Summary'])  # Write the header
    writer.writerows(news_list)  # Write the sorted news data